<a href="https://colab.research.google.com/github/amalp12/MLAI/blob/main/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Imports**

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from google.colab.patches import cv2_imshow
import numpy as np
from torch.utils.tensorboard import SummaryWriter  # to print to tensorboard
import pdb # python debugger https://web.stanford.edu/class/physics91si/2013/handouts/Pdb_Commands.pdf

**Initialise Hyperparameters**

In [3]:
img_dim = 28*28 # in MNIST, the image dimentions are 28*28
d_hidden = 128
g_hidden = 256
learning_rate = 3e-4
# Get CPU in the game
device = "cuda" if torch.cuda.is_available() else "cpu"
number_of_epochs = 50
batch_size = 32
z_dim = 64
fixed_noise = torch.randn((batch_size, z_dim)).to(device)

**Import Data**

Data Nomalization: it is typical for the range of different features in a  dataset to be of different scale. For example the age of a person can typically range from 0-100, wheras the living cost of a person can range from say 1000 - 10000. This difference can become problematic if we are running ML algorthms on top of this dataset, hence we will need to rescale the dataset features to be on a more similar scale.

In [89]:
# Normalization 
# transforms.Compose just clubs all the transforms provided to it. So, all the transforms in the transforms.Compose are applied to the input one by one.
transform = transforms.Compose(
    [
      
      # Convert a PIL Image or numpy.ndarray to tensor. This transform does not support torchscript.
      # Converts a PIL Image or numpy.ndarray (H x W x C) in the range
      # [0, 255] to a torch.FloatTensor of shape (C x H x W) in the range [0.0, 1.0], note that tranforms.ToTensor does this

      transforms.ToTensor(),
      # Normalize a tensor image with mean and standard deviation.
      # modifies the elements ([0,1]  mean)/std_dev  hence turns it into the range [(0-.5)/.5 , (1-.5)/.5] = [-1,1]
      transforms.Normalize((0.5,),(0.5,))
      # to denormalise  ((output*.5)+.5)*255
    ]
)

# Funtion to Denormalize
def denormalize(x):
  # Denormalizeing 
  pixels =  ((x *.5)+.5)*255
  return pixels

# Load Dataset
dataset = datasets.MNIST(root="dataset/", transform=transform,download=True)
# Samples a batch of data from the dataset 
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
# Each line of data contains 1 + 28*28*1 integers, the first one is label of the data and the rest are pixels  denoting intensity from 0 to 255


**View Data**

In [108]:
# Tuple of tensors
view_dataset = datasets.MNIST(root="dataset/" , transform=transform)
test_loader = DataLoader(view_dataset, batch_size=batch_size, shuffle=True)
data = None
label = None
#pdb.set_trace()
for epoch in range(number_of_epochs):
  for batch_index, (real, lab)  in enumerate(test_loader): # _ is label, For gan we are not going to use labels hence we put _
    # Data is provided as an enumerate
    print(f"Enumerate Output Before Reshaping : Index : {batch_index}, Data Shape: {real.shape}, Labels Shape: {lab.shape}") # enumerate outputs the index, the PIL image form of the data, and the label
    # reshaping
    real = real.reshape(32,28,28)

    # After reshaping
    print(f"Enumerate Output After Reshaping : Index : {batch_index}, Data Shape: {real.shape}, Labels Shape :{lab.shape}") # enumerate outputs the index, the PIL image form of the data, and the label

    data = real
    label = lab
    break
  break



# The first column is the label

print(f"Label : {label[0]}")
# The rest of columns are pixels
pixels = data[0]
pixels = np.array(pixels)

# Denormalizeing 
#pixels =  ((pixels *.5)+.5)*255
pixels = denormalize(pixels)

cv2_imshow(pixels)


Enumerate Output Before Reshaping : Index : 0, Data Shape: torch.Size([32, 1, 28, 28]), Labels Shape: torch.Size([32])
Enumerate Output After Reshaping : Index : 0, Data Shape: torch.Size([32, 28, 28]), Labels Shape :torch.Size([32])
Label : 7


**BCE LOSS**

In [7]:
criterion = nn.BCELoss()

**Define Discriminator Loss**

Max [ log (D(x)) + log(1-D(G(z))]

In [107]:
# Not used here, we use BCE Loss
def disc_loss(dx,dgz):
  assert type(dx) == torch.Tensor
  assert type(dgz) == torch.Tensor
  loss_model = torch.log(dx) + torch.log(1-dgz)
  return loss_model 

**Define Generator Loss**

Min [ log(1-D(G(z))] <--> Max [ log (D(G(z)))] 

In [106]:
# Not used here, we use BCE Loss
def gen_loss():
  pass

**Define Discriminator Class**

In [10]:
class Discriminator(nn.Module) :
  def __init__(self, lr):
    super().__init__()

    # takes in image dimentions
    self.hidden =  128
    self.img_dim = 28*28 #784
    
    self.model = nn.Sequential(
        nn.Linear(in_features=self.img_dim, out_features= self.hidden),
        nn.LeakyReLU(),
        nn.Linear(in_features=self.hidden, out_features= 1),
        nn.Sigmoid()
     
    )
    self.optimizer = optim.Adam(params= self.parameters(), lr= lr)
    
  def forward(self,x):
    return self.model(x)

**Define Generator Class**

In [11]:
class Generator(nn.Module) :
  def __init__(self,lr):
    self.hidden =  256
    self.noise_dim = 64
    self.img_dim = 28*28#784
    super().__init__()
    # takes in noise dim
    self.model = nn.Sequential(
        nn.Linear(in_features = self.noise_dim, out_features= self.hidden ), #
        nn.LeakyReLU(0.1),
        nn.Linear(in_features=self.hidden , out_features= self.img_dim),
        nn.Tanh(), # we use tanh to ensure the output of gen is b/w -1 and 1 to  normalize
    )
    self.optimizer = optim.Adam(params= self.parameters(), lr= lr)
    self.loss = nn.BCELoss()
  def forward(self, x):
    return self.model(x)

**Initialzing Discriminator and Generator**

In [12]:
disc = Discriminator(learning_rate).to(device)
gen = Generator(learning_rate).to(device)




**Training** 

In [104]:
for epoch in range(number_of_epochs):
  for batch_index, (real_x, label)  in enumerate(loader): # _ is label, For gan we are not going to use labels hence we put _
    
    real_x = torch.flatten(real_x, start_dim = 1).to(device) # flattenned it into 32 x 784 
    assert batch_size == real_x.shape[0] # Test Line
    


    #### Training the Discriminator

    # Generateing Fake images
    noise = torch.randn(batch_size, z_dim ).to(device) # creating noise in the dimention of batch size x input of generator (z_dim)
    fake_images = gen(noise)
    
    # cv2_imshow( np.resize( fake_images.detach().numpy(), (28,28))) ## Testline - View Images
    
    disc_real_x = disc(real_x) # getting real image outputs
    disc_real_x = torch.flatten(disc_real_x) # flattening the outputs
    
    all_ones = torch.ones_like(disc_real_x) # creating a torch with same dimention as disc real with all values one

    # getting loss between disc real output and all ones
    disc_real_x_loss = criterion(disc_real_x, all_ones)
    
    # getting the flattened version of the generated fake images
    disc_fakes = disc(fake_images)
    disc_fakes = torch.flatten(disc_fakes) # flattening the outputs

    all_zeros = torch.zeros_like(disc_fakes) # creating a torch with same dimention as disc fake with all values zeros
    
    # getting loss between gen fake outputs and all zeros
    disc_fakes_loss = criterion(disc_fakes, all_zeros)
    
    # average fake and real loss
    disc_loss = (disc_real_x_loss + disc_fakes_loss) / 2

    disc.zero_grad() # clearing disc gradients
    # when we use loss.backward(), everything we used to calculate the gradients will be cleared, if we do not want that to happen then use retain graph
    disc_loss.backward(retain_graph = True) # retained because we need to reuse fake_images
    disc.optimizer.step()





    #### Training Generator

    # Getting gake image outputs, when passed through the trained generator
    trained_disc_outputs = disc(fake_images)
    trained_disc_outputs = torch.flatten(trained_disc_outputs) # flattening the outputs
    all_ones = torch.ones_like(trained_disc_outputs) # creating a torch with same dimention as trained gen outputs with all values one
    gen_loss = criterion( trained_disc_outputs, all_ones)
    gen.zero_grad()
    gen_loss.backward()
    gen.optimizer.step()
    

    
    if batch_index == 0:
            print(
                f"Epoch [{epoch}/{number_of_epochs}] Batch {batch_index}/{len(loader)} \
                      Loss D: {disc_loss:.4f}, loss G: {gen_loss:.4f}"
            )

                #pdb.set_trace()
            with torch.no_grad():
                random_noise = torch.randn((batch_size, z_dim)).to(device)
                fake = gen( random_noise).reshape((32,28,28)).numpy()
                data = real_x.reshape(32,28, 28).numpy()
                # denormalize
                fake = denormalize(fake)
                data = denormalize(data)
                #pdb.set_trace()
                print("Real Image")
                cv2_imshow(data[16])          

                print("Fake Image")
                cv2_imshow(fake[16])   

                

                
    
    

Epoch [0/50] Batch 0/1875                       Loss D: 0.6482, loss G: 1.0050
Real Image


Fake Image


Epoch [1/50] Batch 0/1875                       Loss D: 0.5028, loss G: 1.0977
Real Image


Fake Image


Epoch [2/50] Batch 0/1875                       Loss D: 0.5400, loss G: 1.1667
Real Image


...

Fake Image


Epoch [48/50] Batch 0/1875                       Loss D: 0.5398, loss G: 1.0733
Real Image


Fake Image


Epoch [49/50] Batch 0/1875                       Loss D: 0.6245, loss G: 1.2126
Real Image


Fake Image
